In [ ]:
from config.database import collection_airports_cache_legacy, collection_weather_cache_legacy, airport_bulk_collection_uj, db_UJ, collection_searchTrackUsers, collection_flights
from core.weather_parse import Weather_parse

find_crit = {"code": 'EWR'}
return_crit = {}
# return_crit = {'weather':1,'code':1,'_id':0}
res = collection_weather_cache_legacy.find_one(find_crit, return_crit)

code = res.get('code') if res else None

if res:
    weather_raw = res.get('weather')
    # HTML injection to color code the weather data
    wp = Weather_parse()
    weather = wp.html_injected_weather(weather_raw=weather_raw)
    weather.update({'code':code})       # add airport code to the weather dict
    # print('res weather',weather )
# res

In [ ]:
db_UJ.list_collection_names()   # get all collections of the db


In [ ]:
search_index_collection = db_UJ['search_index']

# start fresh reset test collection replace from scratch
# db_UJ['search_index_test'].delete_many({})
# search_index_collection.aggregate([{'$out': 'search_index_legacy_backup'}])

new_airport_cache_collection = db_UJ['airport-cache-test']

search_index_collection_test = db_UJ['search_index_test']
ctrs = db_UJ['test_rst']   # create/get a collection

suggestions_cache_collection = db_UJ['suggestions-cache']
suggestions_test_collection = db_UJ['suggestions-cache-test-refills']


In [ ]:
flightID = 'AAL'

find_crit = {
    "versions.version_created_at": {"$exists": True},
    "flightID": {"$regex": flightID}
}
return_crit = {'flightID':1,'_id':0}
    # TODO search suggestions: See here we can add fields with ICAO/IATA airline codes as customary so for commuteair, gojet can do UA and endevor can do DL and so on?

# flight_docs = collection_flights.find(find_crit, return_crit).limit(3000)
# flight_docs = list(flight_docs)
# flight_docs
collection_flights.count_documents(find_crit, )



In [ ]:
collection_flights.find_one({'flightID': 'GJS408A'})

In [ ]:
ICAO_airport_code = 'KEWR'
airport_bulk_collection_uj.find_one({'icao': ICAO_airport_code})


In [ ]:
from core.search.search_interface import ExhaustionCriteria


ec = ExhaustionCriteria()
a = await ec.faa_airport_info_fetch("KEWR")
a


In [ ]:
ICAO_airport_code = 'KEWR'
find_crit = {'.country_code': 'US'}
doc = airport_bulk_collection_uj.find_one({'icao': ICAO_airport_code})
doc


In [ ]:
find_crit = {'type':'flight'}
suggestions_cache_collection.find_one(find_crit)
# suggestions_cache_collection.insert_many(update_docs)
# suggestions_cache_collection.count_documents(find_crit)


In [ ]:
from services.search_service import get_search_suggestions_service
a = await get_search_suggestions_service(email='',query='yow')


In [ ]:
import requests
from core.api.source_links_and_api import Source_links_and_api


ICAO_airport_code = 'KEWR'
faa_weather_link = Source_links_and_api().airport_info_faa(ICAO_airport_code)

response = requests.get(faa_weather_link)
data = response.content.decode('utf-8')

import json
data = json.loads(data)

data



In [ ]:
find_crit = {    "$or": [
        {"country_code": {"$type": "number"}},
        # {"icao": {"$type": "number"}},
        # {"iata": {"$type": "number"}},
        # {"airport": {"$type": "number"}},
        # {"latitude": {"$type": "number"}},
        # {"longitude": {"$type": "number"}}
    ]}
update_crit = {'$set':{'country_code':''}}
# airport_bulk_collection_uj.update_many(find_crit,update_crit)
# list(docs)

In [ ]:
airport_bulk_collection_uj.find_one({'icao':'CYOW'})
# airport_bulk_collection_uj.count_documents(find_crit)

In [ ]:
a

In [ ]:
from config.database import gate_rows_collection

gate_rows_collection.find_one({'Gate':'C101'})
# 69128692d558bf7662b4502c

In [ ]:
from pymongo import UpdateOne


find_crit = {'type':'gate'}
# find_crit = {}
gate_docs = suggestions_cache_collection.find(find_crit)
update_operations = []
for i in gate_docs:
    gate_source_doc = gate_rows_collection.find_one({'Gate':i.get('display')}, {'_id':1})

    if gate_source_doc:
        display = i.get('display')
        gate_id = gate_source_doc.get('_id')

        if display:
            new_display = f"EWR - {display} Departures"
        
        new_display_similarity = [f"EWR - {d} Departures" for d in i.get("displaySimilarity", [])]
        update = {
            "display": new_display, 
            "displaySimilarity": new_display_similarity,
            "reference_id": gate_id
        }
        update_operations.append(
            UpdateOne(
                {"_id": i["_id"]},
                {"$set": update}
            )
        )

# suggestions_cache_collection.bulk_write(update_operations)






In [ ]:
from bson import ObjectId

find_crit = {'display':{'$regex':'DCA'}}
doc = suggestions_cache_collection.find_one(find_crit)
r = doc['referenceId']
new_airport_cache_collection.find_one({'_id':r})
'690abf879832368ab25c25c2'

In [ ]:
doc

In [ ]:
from bson import ObjectId


find_crit = {'_id': ObjectId('6912b9cd9922b1ce4f9f3f6a')}
update_crit = {'displaySimilarity': ['SCE / KUNV - University Park Airport', 'KSCE']}


find_crit = {}
suggestions_cache_collection.find_one(find_crit)
# suggestions_cache_collection.update_one(find_crit, {'$set':update_crit})

In [ ]:
""" Proof of concept(Airports) for suggestions cache collection migration from legacy SIC to new suggestions cache collection - Just airports. """

from pprint import pprint


find_crit = {'airport_st':{"$exists":True}}
# find_crit = {}
sic_gate_docs = search_index_collection.find(find_crit)

non_existant_codes = set()
update_docs = []
# SIC to Suggestions cache migration.
        #  Flow is: SIC-og r_id(airport_st- airports only) --> r_id to _id of airports cache legacy --> iata of airport cache legacy to weather `code`(also an iata)
for SIC_og_doc in sic_gate_docs:
    # Transfer the whole SIC doc with new airportReferenceId to suggestions_cache_collection - account for Iata that maybe different from the icao if last 3 chars are different.
    
    if not SIC_og_doc:
        print('no SIC_og_doc', SIC_og_doc)
        continue
    SIC_bson_r_id = SIC_og_doc['r_id']      # to get airport_id from legacy cache

    airport_cache_legacy_doc = collection_airports_cache_legacy.find_one({'_id':SIC_bson_r_id})
    weather_doc = collection_weather_cache_legacy.find_one({'airport_id': SIC_bson_r_id})
    if not weather_doc:
        print('no weather doc', SIC_og_doc)
        continue
    else:
        weather_doc_icao = weather_doc.get('ICAO')
        if not weather_doc_icao:
            print('no weather doc icao', SIC_og_doc)


    # airport_cache_iata = airport_cache_legacy_doc['code']     # probably dont need this because were already getting this data from weather doc.
    # Preparing new weather cache doc with SIC legacy doc to suggestions cache.
    new_weather_cache_doc = new_airport_cache_collection.find_one({'ICAO': weather_doc_icao})
    if not new_weather_cache_doc:
        print('no new_weather_cache_doc', SIC_og_doc)
        continue
    SIC_og_doc['r_id'] = new_weather_cache_doc['_id']
    new_suggestions_cache_structure = {
        # 'id': 'unique_id',
        'type': 'airport',  # 'airport|flight|gate',  # Standardized types
        'referenceId': new_weather_cache_doc['_id'], # bson id - associated collection id to fetch detailed assocaited type data i.e airport, flight, gate,
        'display': SIC_og_doc['airport_st'],           #'str - User-friendly display text',
        'displaySimilarity': [SIC_og_doc['airport_st']], # "str|array - Text used for fuzzy matching",  # can be an array for multiple search terms
        'popularityScore': SIC_og_doc['ph'],
    }

    submits = SIC_og_doc.get('submits')
    if submits:
        new_suggestions_cache_structure['submitTimestamps'] = submits
    update_docs.append(new_suggestions_cache_structure)


    # airportReferenceId = new_weather_cache_doc['_id']         # This goes into the the suggestions cache as airportReferenceId along with SIC legacy doc as is.
    

# for i in update_docs:
    # if i['ICAO] in weather_legacy:
#     update_operations.append(
#         UpdateOne({'flightID': flightNumber},
#                   {'$set': {'flightID': flightNumber}},
#                   upsert=True
#                   )
#     ),
# result = collection_backup.bulk_write(update_operations)


In [ ]:
""" Proof of concept(Flights) for suggestions cache collection migration from legacy SIC to new suggestions cache collection - Just flights. """

from pprint import pprint


find_crit = {'fid_st':{"$exists":True}}
# find_crit = {}
sic_gate_docs = search_index_collection_test.find(find_crit)

non_existant_codes = set()
update_docs = []
# SIC to Suggestions cache migration.
for SIC_og_doc in sic_gate_docs:
    new_suggestions_cache_structure = {
        # 'id': 'unique_id',
        'type': 'flight',  # 'airport|flight|gate',  # Standardized types
        'reference_id': SIC_og_doc['r_id'], # bson id - associated collection id to fetch detailed assocaited type data i.e airport, flight, gate,
        'display': SIC_og_doc['fid_st'],           #'str - User-friendly display text',
        'displaySimilarity': [SIC_og_doc['fid_st']], # "str|array - Text used for fuzzy matching",  # can be an array for multiple search terms
        'popularityScore': SIC_og_doc['ph'],
    }

    submits = SIC_og_doc.get('submits')
    if submits:
        new_suggestions_cache_structure['submitTimestamps'] = submits
    update_docs.append(new_suggestions_cache_structure)


In [ ]:
""" Proof of concept(Gate) for suggestions cache collection migration from legacy SIC to new suggestions cache collection - Just flights. """

from pprint import pprint


find_crit = {'Terminal/Gate':{"$exists":True}}
# find_crit = {}
sic_gate_docs = search_index_collection.find(find_crit)

non_existant_codes = set()
update_docs = []
# SIC to Suggestions cache migration.
for SIC_og_doc in sic_gate_docs:
    new_suggestions_cache_structure = {
        # 'id': 'unique_id',
        'type': 'gate',  # 'airport|flight|gate',  # Standardized types
        'reference_id': SIC_og_doc.get('r_id'), # bson id - associated collection id to fetch detailed assocaited type data i.e airport, flight, gate,
        'display': SIC_og_doc['Terminal/Gate'],           #'str - User-friendly display text',
        'displaySimilarity': [SIC_og_doc['Terminal/Gate']], # "str|array - Text used for fuzzy matching",  # can be an array for multiple search terms
        'popularityScore': SIC_og_doc['ph'],
    }

    submits = SIC_og_doc.get('submits')
    if submits:
        new_suggestions_cache_structure['submitTimestamps'] = submits
    update_docs.append(new_suggestions_cache_structure)



In [ ]:
from config.database import collection_flights


# find_crit = {'Terminal/Gate':{"$exists":True}}
find_crit = {}
# flight_docs = search_index_collection_test.find_one(find_crit)
# search_index_collection_test.find_one(find_crit)
search_index_collection_test.count_documents(find_crit)
# collection_flights.find_one({'_id': flight_docs})

# search_index_collection_test.delete_many(find_crit)


In [ ]:
# suggestions_cache_collection.insert_many(update_docs)
suggestions_cache_collection.count_documents({})

In [ ]:
# Add docs to airport_cache_collection w weather:


bulk_doc = airport_bulk_collection_uj.find_one({'icao':weather_doc_icao})         # probs dont need this too. Bulk is only used as airport ICAO catcher - Its static. Accuracy maybe a problem

if bulk_doc:
    ICAO_airport_code = bulk_doc.get('icao')
    IATA_airport_code = bulk_doc.get('iata')
    airportName = bulk_doc.get('airport')
    regionName = bulk_doc.get('region_name')
    countryCode = bulk_doc.get('country_code')
else:
    print('no bulk doc', SIC_og_doc)

if not weather_doc or not weather_doc.get('code'):
    non_existant_codes.add(non_existant_codes['_id'])
else:
    new_weather_doc = {
        'IATA': IATA_airport_code,
        'ICAO': ICAO_airport_code,       
        'airportName': airportName,
        'regionName': regionName,
        'countryCode': countryCode,

        'weather': weather_doc.get('weather', {}),
        
        # 'suggestionsReferenceId': sic_doc['_id'],     # originally a consideration but it doesn't make sense to have two way reference ID, just have one on suggestions_cache and direct it to airport_cache
    }
    update_docs.append(new_weather_doc)
    # Youre not going to delete search index collection
    # search_index_collection_test.delete_one({'_id':sic_doc['_id']})


In [ ]:
find_crit = {'airportDisplayTerm': {'$regex': "ANE"}}
# set_crit = {'$set':{'airportDisplayTerm':'SCE / KUNV - University Park Airport'}}
# search_index_collection_test.update_one(find_crit,set_crit)
# search_index_collection_test.delete_one(find_crit)
# collection.update_one({'flightID':'GJS123'},{'$set':{'count':7.92193}})


In [ ]:
from schema.mongo_schema import MongoStructure
ms = MongoStructure()
ms.get_structure(collection=search_index_collection_test)
# list(ctrs.find({}).limit(10))


In [ ]:
# Aviation weather response parsing for airport_cache_collection
from core.weather_fetch import Singular_weather_fetch
from models.model import AirportCache
import requests

def parse_aviation_weather_response(api_data):
    """Parse the text-based API response from aviationweather.gov"""
    parsed = {}
    
    lines = api_data.strip().split('\n')
    for line in lines:
        line = line.strip()
        if not line:
            continue
            
        # Parse key-value pairs (looking for specific keys)
        if ':' in line:
            key, value = line.split(':', 1)
            key = key.strip()
            value = value.strip()
            
            if key == 'IATA':
                parsed['IATA'] = value if value != '-' else None
            elif key == 'ICAO':
                parsed['ICAO'] = value
            elif key == 'Name':
                parsed['airportName'] = value
            elif key == 'State':
                parsed['regionName'] = value
            elif key == 'Country':
                parsed['countryCode'] = value
            # elif key == 'Elevation':
            #     parsed['elevation'] = value
            # elif key == 'Latitude':
            #     parsed['latitude'] = value
            # elif key == 'Longitude':
            #     parsed['longitude'] = value
    
    parsed['weather']= {}
    return parsed


async def x(ICAO_airport_code):
    bulk_doc = airport_bulk_collection_uj.find_one({'icao':ICAO_airport_code})
    if not bulk_doc:
        response = requests.get(f'https://aviationweather.gov/api/data/airport?ids={ICAO_airport_code}')
        
        data = response.content.decode('utf-8')
        if not data:
            return
        if isinstance(data,dict) and data.get('status') == 'error':
            print(data['status'])
            return
        if 'error' in data:
            return
        update_doc = parse_aviation_weather_response(data)

        # print(json.loads(response.content))
    else:
        update_doc = {
            'IATA': bulk_doc['iata'],
            'ICAO': bulk_doc['icao'],
            'airportName': bulk_doc['airport'],
            'regionName': bulk_doc['region_name'],
            'countryCode': bulk_doc['country_code'],
            'weather': {}
        }
    AirportCache(**update_doc)

    swf  = Singular_weather_fetch()
    weather_dict = await swf.async_weather_dict(update_doc['ICAO'])
    update_doc['weather'] = weather_dict
    return update_doc
# await x('KSMQ')
# await x('C-29')

In [ ]:

# raw searches match icao/iata find them in legacy iata 
    # if exists, use iata to get associated icao using bulk_airport_collection with us as region.
    # if found, and should be there since its in legacy cache, then check SIC if it already exists, then add to its submits
    # then add this new_airport_cache_collection using x(icao)

from bson import ObjectId

count = 0
for rst_doc in ctrs.find():
    code = rst_doc['rst'].upper()
    code = code.strip()
    if code.isdigit():      # Probably flight number
        continue
    elif len(code) == 3:    # probably IATA airport code
        # print(code)
        pass
        # iata = code
        # # get iata from legacy using this iata, if exists use its id to find icao within weather_collection_legacy
        # found_airport= collection_airports_cache_legacy.find_one({'code':iata})     # from legacy find iata code
        # if found_airport:
        #     ref_id = found_airport['_id']
        #     doc = collection_weather_cache_legacy.find_one({'airport_id': ref_id}, {'ICAO':1})
        #     if not doc:
        #         print(iata)
        #         rst_submits = rst_doc['submits']
        #         print(rst_submits)
        #     else:
        #         ICAO_airport_code= doc.get('ICAO')
        #         y = new_airport_cache_collection.find_one({'ICAO':ICAO_airport_code})
        #         if not y:
        #             print(ICAO_airport_code)
        # elif not found_airport:
        #     pass
        #     print('assocaited iata not found in legacy airport cache', iata)
    elif len(code) == 4 and code.isalpha():        # probs ICAO airport code
        icao = code
        found_airport_in_new_cache= new_airport_cache_collection.find_one({'ICAO':icao})
        if not found_airport_in_new_cache:
            # if not in new_cache find it in bulk
            # bulk_doc = airport_bulk_collection_uj.find_one({'icao':icao})
            
            # print(code)
            # update_doc = await x(icao)
            # if update_doc:
                # print(update_doc)



                # new_airport_cache_collection.insert_one(update_doc)
            # ctrs.delete_one({'_id': ObjectId(rst_doc['_id'])})
            pass
    else:
        print(code)
    #             # new_airport_cache_collection.insert_one(update_doc)
    #         else:
    #             print('already in new_airport_cache', icao)
    #         print(update_doc, icao)
    #         update_doc = await x(icao)
    #         if update_doc:
    #         else:
    #             print('not inserted', icao)

In [ ]:

count_crit = {'popularityScore':{"$exists":True}}       # return ones with popularity hits
return_crit = {'submitTimestamps':0}       # We dont need submits since its not used in search interface
# suggestions_cache_collection.count_documents(count_crit)
a = suggestions_cache_collection.find(count_crit, return_crit).sort('popularityScore',-1)     # Reverse sort
a = list(a)


In [ ]:
new_airport_cache_collection.find_one({'ICAO':'KEWR'})
# collection_airports_cache_legacy.find_one({'_id':'r_id'})
# search_index_collection_test.find_one()
# search_index_collection_test.count_documents({})
# airport_bulk_collection_uj.find_one({'icao':'KM91'})
# returns = await x('CYYZ')
# returns

In [ ]:
from schema.mongo_schema import MongoStructure
ms = MongoStructure()
# ms.get_structure(collection=ctrs)
# ms.get_structure(collection=search_index_collection_test)
ms.get_structure(collection=collection_airports_cache_legacy)
# ms.get_structure(collection=collection_weather_cache_legacy)
ms.get_structure(collection=airport_bulk_collection_uj)


# list(ctrs.find({}).limit(10))
# search_index_collection_test.find_one({'_id':next(iter(non_existant_codes))})
a = collection_airports_cache_legacy.find({'name': { '$exists': False } })
list(a)


In [ ]:
update_docs


In [ ]:
# start fresh reset test collection replace from scratch
db_UJ['airport-weather-test'].delete_many({})
db_UJ['airport-weather'].aggregate([{'$out': 'airport-weather-test'}])


In [ ]:
new_collection = db_UJ['airport-weather-test']

for weather_doc in collection_weather_cache_legacy.find({}):
    code = weather_doc.get('code')
    
    if not code:
        print('no doc found')
        break
    
    new_weather_doc = {
        'IATA': code.upper(),
        'ICAO': original_doc.get('ICAO'),
        'airportName': airport_cache_legacy_doc.get('airport'),
        'regionName': airport_cache_legacy_doc.get('region_name'),
        'airportBulkCollectionReferenceId': airport_cache_legacy_doc.get('_id'),
        'weatherData': original_doc.get('weather', {}),
        # 'original_document_id': original_doc.get('_id'),
        # 'migrated_at': datetime.now(timezone.utc)
    }


In [ ]:
# collection_weather_cache_legacy.aggregate([{'$out': 'weather_backup'}])
# Get all US airports from bulk collection for quick lookup
from pymongo import InsertOne


us_airports = {}
for airport in airport_bulk_collection_uj.find({'country_code': 'US'}):
    us_airports[airport['iata']] = airport

operations = []
migrated_count = 0

for original_doc in collection_weather_cache_legacy.find():
    code = original_doc.get('code')
    
    if not code or code not in us_airports:
        print(f"Skipping document with code: {code}")
        continue
        
    airport_cache_legacy_doc = us_airports[code]
    
    new_weather_doc = {
        'IATA': code.upper(),
        'ICAO': original_doc.get('ICAO'),
        'airportName': airport_cache_legacy_doc.get('airport'),
        'regionName': airport_cache_legacy_doc.get('region_name'),
        'airportBulkCollectionReferenceId': airport_cache_legacy_doc.get('_id'),
        'weatherData': original_doc.get('weather', {}),
        # 'original_document_id': original_doc.get('_id'),
        # 'migrated_at': datetime.now(timezone.utc)
    }
    
    operations.append(InsertOne(new_weather_doc))
    migrated_count += 1
    
    # Execute in batches of 100
    if len(operations) >= 400:
        collection_weather_cache_legacy.bulk_write(operations)
        operations = []
        print(f"Migrated batch of 400 documents...")

# Insert any remaining documents
if operations:
    collection_weather_cache_legacy.bulk_write(operations)

print(f"Bulk migration completed: {migrated_count} documents migrated")


In [ ]:
xx = [i.get('code') for i in weather_collection.find({}) if i.get('ICAO')]

In [ ]:
my_list = xx
duplicates = [x for x in set(xx) if xx.count(x) > 1]
print(duplicates)  # Output: [2, 4, 6]

In [ ]:
x = list(collection_searchTrackUsers.find({}))
x


In [ ]:
for original_doc in weather_collection.find():
    code = original_doc.get('code')
    print(code)
    break


In [ ]:
# res = collection_weather_cache_legacy.find_one(find_crit, return_crit)
res = airport_bulk_collection_uj.find_one({'iata':'EWR'})


res


In [ ]:
err = {'metar' : '<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN" "http://www.w3.org/TR/html4/loose.dtd">\n<HTML><HEAD><META HTTP-EQUIV="Content-Type" CONTENT="text/html; charset=iso-8859-1">\n<TITLE>ERROR: The request could not be satisfied</TITLE>\n</HEAD><BODY>\n<H1>403 ERROR</H1>\n<H2>The request could not be satisfied.</H2>\n<HR noshade size="1px">\nRequest blocked.\nWe can\'t connect to the server for this app or website at this time. There might be too much traffic or a configuration error. Try again later, or contact the app or website owner.\n<BR clear="all">\nIf you provide content to customers through CloudFront, you can find steps to troubleshoot and help prevent this error by reviewing the CloudFront documentation.\n<BR clear="all">\n<HR noshade size="1px">\n<PRE>\nGenerated by cloudfront (CloudFront)\nRequest ID: qECkcTBSXsM5q9-WJ9VWdpxXd6lODUh_LUsuo_SOXjCDzGZE7E7Rfg==\n</PRE>\n<ADDRESS>\n</ADDRESS>\n</BODY></HTML>'}
x['metar'] = err['metar']


In [ ]:
from core.tests.broad_test import Broad_test


bt = Broad_test()
bt.weather_test()

In [ ]:
find_crit = {"code": 'PHL'}
return_crit = {'weather':1,'code':1,'_id':0}
res = collection_weather_cache_legacy.find_one(find_crit)
res

In [ ]:
collection_airports_cache_legacy.find_one({"code": 'LIT'})
# collection_airports_cache.find_one({"_id": '66176711170d1d57a24dfb23'})

In [ ]:
from core.weather_fetch import Singular_weather_fetch


airportCode = 'KMEM'
swf  = Singular_weather_fetch()
weather_dict = await swf.async_weather_dict(airportCode)
weather_dict